In [36]:
import happybase
import pandas as pd
import pymysql
from pymysql import MySQLError
from datetime import datetime
import time
import numpy as np
import sys
sys.path.append("..")
from database.redis import *
from main import *
from database.crud import *
from database.models import *

In [15]:
datacoder = DateEncoder()
date = datacoder.encode(datetime.now(),"HOUR")
date

2022070319

In [16]:
# con = happybase.Connection('10.126.62.37',9494, autoconnect=False)
con = happybase.Connection('172.31.41.139',9090, autoconnect=False)
# con = happybase.Connection('172.31.41.139',9090,transport='buffered',protocol='compact')
con.open()  # 打开传输
print(con.tables())  # 输出所有表名
# con.close()  # 关闭传输

table = con.table('hbase_info_1_2')

[b'exp3_hbase0', b'exp3_hbase1', b'exp3_hbase2', b'exp3_hbase3', b'hbase_info_1', b'hbase_info_1_2', b't1']


In [17]:
# count = 0
# row_start、row_stop
hbase_line_count=0
for key,value in table.scan():
    line_dic = {}
    line_dic["time"] = int(key.decode())
    line_dic["pk"] = int(value[b'cf1:Pk'].decode())
    line_dic["memory_total"] = int(value[b'cf1:Memory_total'].decode())
    line_dic["memory_used"] = int(value[b'cf1:Memory_used'].decode())
    line_dic["hdd_total"] = int(value[b'cf1:Hdd_total'].decode())
    line_dic["hdd_used"] = int(value[b'cf1:Hdd_used'].decode())
    line_dic["cpu"] = float(value[b'cf1:Cpu'].decode())
    line_dic["network_in"] = int(value[b'cf1:Network_in'].decode())
    line_dic["network_out"] = int(value[b'cf1:Network_out'].decode())
    for i in range(int(value[b'cf1:Gpu_count'].decode())):
        line_dic[f"gpu_{i}_used"] = int(value[(f'cf1:Gpu_{i}_used').encode('utf-8')].decode())
    for i in range(int(value[b'cf1:Gpu_count'].decode())):
        line_dic[f"gpu_{i}_total"] = int(value[(f'cf1:Gpu_{i}_total').encode('utf-8')].decode())
    
    if hbase_line_count==0:
        hbase_data_df = pd.DataFrame([line_dic])
    else:
        hbase_data_df = hbase_data_df.append(line_dic,ignore_index=True)
    hbase_line_count = hbase_line_count+1
    
    # gpu_0_used = int(value[b'cf1:Gpu_0_used'].decode())
    # gpu_1_used = int(value[b'cf1:Gpu_1_used'].decode())
    # gpu_2_used = value[b'cf1:Gpu_2_used'].decode())
    # gpu_3_used = value[b'cf1:Gpu_3_used'].decode())
    # line_dic = {"time":int(key.decode()),
    #             "memory_total":memory_total,
    #             "memory_used":memory_used,
    #             "hdd_total":hdd_total,
    #             "hdd_used":hdd_used,
    #             "cpu":cpu,
    #             "network_in":network_in,
    #             "network_out": network_out,
    #             "gpu_0_used":gpu_0_used,"gpu_1_used":gpu_1_used}

    # hbase_data_df.append({"time":time,"memory_total":memory_total,"memory_used":memory_used,"hdd_total":hdd_total,
    #                                     "hdd_used":hdd_used,"cpu":cpu,"network_in":network_in,"network_out": network_out,
    #                                     "gpu_0_used":gpu_0_used,"gpu_1_used":gpu_1_used,"gpu_2_used":gpu_2_used,"gpu_3_used":gpu_3_used}
    # #                                     ,ignore_index=True)
    # hbase_data_df =hbase_data_df.append({"time":time,"memory_total":memory_total,"memory_used":memory_used,"hdd_total":hdd_total,
    #                                     "hdd_used":hdd_used,"cpu":cpu,"network_in":network_in,"network_out": network_out,
    #                                     "gpu_0_used":gpu_0_used,"gpu_1_used":gpu_1_used}
    #                                     ,ignore_index=True)
    # print(key,value)
    

In [18]:
gpu_num = int((len(hbase_data_df.columns)-9)/2)
print(gpu_num)

4


In [19]:
hbase_data_df

,time,pk,memory_total,memory_used,hdd_total,hdd_used,cpu,network_in,network_out,gpu_0_used,gpu_1_used,gpu_2_used,gpu_3_used,gpu_0_total,gpu_1_total,gpu_2_total,gpu_3_total
0,1.656257e+09,1.0,33140768.0,7202140.0,7680485.0,7202140.0,55.9,8.791300e+10,5.870014e+10,11105.0,10.0,3621.0,3701.0,12196.0,12196.0,12196.0,12196.0
1,1.656257e+09,1.0,33139644.0,7202140.0,7680485.0,7202140.0,54.6,8.791302e+10,5.870015e+10,11105.0,10.0,3621.0,3701.0,12196.0,12196.0,12196.0,12196.0
2,1.656257e+09,1.0,33144732.0,7202140.0,7680485.0,7202140.0,55.4,8.791303e+10,5.870016e+10,11105.0,10.0,3621.0,3701.0,12196.0,12196.0,12196.0,12196.0
3,1.656257e+09,1.0,33128472.0,7202140.0,7680485.0,7202140.0,54.6,8.791304e+10,5.870017e+10,11105.0,10.0,3621.0,3701.0,12196.0,12196.0,12196.0,12196.0
4,1.656257e+09,1.0,33124864.0,7202140.0,7680485.0,7202140.0,56.7,8.791305e+10,5.870018e+10,11105.0,10.0,3621.0,3701.0,12196.0,12196.0,12196.0,12196.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
343,1.656260e+09,1.0,29903728.0,7202134.0,7680485.0,7202134.0,29.9,8.791769e+10,5.870467e+10,11105.0,10.0,10.0,10.0,12196.0,12196.0,12196.0,12196.0
344,1.656260e+09,1.0,29921140.0,7202134.0,7680485.0,7202134.0,29.4,8.791770e+10,5.870468e+10,11105.0,10.0,10.0,10.0,12196.0,12196.0,12196.0,12196.0
345,1.656260e+09,1.0,29956492.0,7202134.0,7680485.0,7202134.0,25.6,8.791771e+10,5.870469e+10,11105.0,10.0,10.0,10.0,12196.0,12196.0,12196.0,12196.0
346,1.656260e+09,1.0,29896916.0,7202134.0,7680485.0,7202134.0,35.0,8.791773e+10,5.870470e+10,11105.0,10.0,10.0,10.0,12196.0,12196.0,12196.0,12196.0


In [20]:
for i in range(len(hbase_data_df)):
    hbase_data_df.loc[i,'datetime']=datetime.utcfromtimestamp(hbase_data_df.loc[i,'time']).strftime("%Y:%m:%d") # "%Y:%m:%d %H:%M:%S"
    hbase_data_df.loc[i,'hour_code']=datacoder.encode(datetime.utcfromtimestamp(hbase_data_df.loc[i,'time']),flag="HOUR")
    if hbase_data_df.loc[i,'gpu_0_used']/16117 <=0.2 or hbase_data_df.loc[i,'gpu_1_used']/16117 <=0.2:
        hbase_data_df.loc[i,'idle_or_not']=1
    else:
        hbase_data_df.loc[i,'idle_or_not']=0

In [21]:
hbase_data_df.head()

,time,pk,memory_total,memory_used,hdd_total,hdd_used,cpu,network_in,network_out,gpu_0_used,gpu_1_used,gpu_2_used,gpu_3_used,gpu_0_total,gpu_1_total,gpu_2_total,gpu_3_total,datetime,hour_code,idle_or_not
0,1.656257e+09,1.0,33140768.0,7202140.0,7680485.0,7202140.0,55.9,8.791300e+10,5.870014e+10,11105.0,10.0,3621.0,3701.0,12196.0,12196.0,12196.0,12196.0,2022:06:26,2.022063e+09,1.0
1,1.656257e+09,1.0,33139644.0,7202140.0,7680485.0,7202140.0,54.6,8.791302e+10,5.870015e+10,11105.0,10.0,3621.0,3701.0,12196.0,12196.0,12196.0,12196.0,2022:06:26,2.022063e+09,1.0
2,1.656257e+09,1.0,33144732.0,7202140.0,7680485.0,7202140.0,55.4,8.791303e+10,5.870016e+10,11105.0,10.0,3621.0,3701.0,12196.0,12196.0,12196.0,12196.0,2022:06:26,2.022063e+09,1.0
3,1.656257e+09,1.0,33128472.0,7202140.0,7680485.0,7202140.0,54.6,8.791304e+10,5.870017e+10,11105.0,10.0,3621.0,3701.0,12196.0,12196.0,12196.0,12196.0,2022:06:26,2.022063e+09,1.0
4,1.656257e+09,1.0,33124864.0,7202140.0,7680485.0,7202140.0,56.7,8.791305e+10,5.870018e+10,11105.0,10.0,3621.0,3701.0,12196.0,12196.0,12196.0,12196.0,2022:06:26,2.022063e+09,1.0


In [22]:
df_expenditure_mean = hbase_data_df.groupby(['hour_code']).mean() # 计算每一组的平均值
df_expenditure_sum = hbase_data_df.groupby(['hour_code']).sum() # 计算每一组的平均值
df_expenditure_count = hbase_data_df.groupby(['hour_code']).count() # 计算每一组的数量
# 通过这种方式来计算所需要各种值，包括平均值和和值
hbase_data_df = pd.DataFrame(columns={"identity","created_time","server","hdd_used_avg","network_up_total",
                                        "network_down_total","memory_avg","gpu_memory_avg","idle_rate"})

In [25]:
df_expenditure_mean.index

Float64Index([2022062615.0, 2022062616.0], dtype='float64', name='hour_code')

In [26]:
df_expenditure_sum.index

Float64Index([2022062615.0, 2022062616.0], dtype='float64', name='hour_code')

In [44]:
session = DBSession()
hour_crud = HourInfoCrud(session)
day_crud = DayInfoCrud(session)
week_crud = WeekInfoCrud(session)
month_crud = MonthInfoCrud(session)
year_crud = YearInfoCrud(session)
re_week_crud = ReportWeekCrud(session)
re_month_crud = ReportMonthCrud(session)
re_year_crud = ReportYearCrud(session)
hbase2mysql_count=0
for index in df_expenditure_mean.index:
    to_mysql_dic = {}
    to_mysql_dic["identity"]=int(index)
    to_mysql_dic["created_time"]=datetime.now()
    to_mysql_dic["server"]=int(df_expenditure_mean.loc[index,"pk"])
    to_mysql_dic["hdd_used_avg"]=int(df_expenditure_mean.loc[index,"hdd_used"])
    to_mysql_dic["network_up_total"]=int(df_expenditure_sum.loc[index,"network_in"])
    to_mysql_dic["network_down_total"]=int(df_expenditure_sum.loc[index,"network_out"])
    to_mysql_dic["memory_avg"]=int(df_expenditure_mean.loc[index,"memory_used"])
    gpu_avg=0
    for i in range(gpu_num):
        gpu_avg+= df_expenditure_mean.loc[index,f"gpu_{i}_used"]
    to_mysql_dic["gpu_memory_avg"] = gpu_avg
    to_mysql_dic["idle_rate"] = df_expenditure_sum.loc[index,"idle_or_not"]/df_expenditure_count.loc[index,"idle_or_not"]


    print("输入数据"+str(hbase2mysql_count))
    hbase2mysql_count = hbase2mysql_count + 1
    
    hour2day_identity = datacoder.encode(datacoder.decode(int(index),flag=ReportType.HOUR),flag=ReportType.DAY)
    hour2week_identity = datacoder.encode(datacoder.decode(int(index),flag=ReportType.HOUR),flag=ReportType.WEEK)
    hour2month_identity = datacoder.encode(datacoder.decode(int(index),flag=ReportType.HOUR),flag=ReportType.MONTH)
    hour2year_identity = datacoder.encode(datacoder.decode(int(index),flag=ReportType.HOUR),flag=ReportType.YEAR)
    
    cur_server_pk = int(df_expenditure_mean.loc[index,"pk"])
    day_or_none = day_crud.find_one(identity=hour2day_identity,server=cur_server_pk)

    if day_or_none is None:
        week_or_none = week_crud.find_one(identity=hour2week_identity,server=cur_server_pk)
        if week_or_none is None:
            week_report_or_none = week_crud.find_one(identity=hour2week_identity)
            if week_report_or_none is None:
                re_week_info = ReportWeek(identity=hour2week_identity,
                                            created_time=datetime.now())
                new_re_week_crud = re_week_crud.add_from_model(re_week_info)
                week_report_id = new_re_week_crud.pk
                week_info = WeekInfo(identity=hour2week_identity,
                                    created_time=datetime.now(),
                                    server = cur_server_pk,
                                    report_id = new_re_week_crud.pk)
                new_week_info = week_crud.add_from_model(week_info)
            else:
                week_report_id = week_report_or_none.pk
                week_info = WeekInfo(identity=hour2week_identity,
                                    created_time=datetime.now(),
                                    server = cur_server_pk,
                                    report_id = week_report_or_none.pk)
                new_week_info = week_crud.add_from_model(week_info)

        month_or_none = month_crud.find_one(identity=hour2month_identity,server=cur_server_pk)
        if month_or_none is None:
            month_report_or_none = month_crud.find_one(identity=hour2month_identity)
            if month_report_or_none is None:
                re_month_info = ReportMonth(identity=hour2month_identity,
                                            created_time=datetime.now())
                new_re_month_crud = re_month_crud.add_from_model(re_month_info)
                month_report_id = new_re_month_crud.pk
                month_info = MonthInfo(identity=hour2month_identity,
                                    created_time=datetime.now(),
                                    server = cur_server_pk,
                                    report_id = new_re_month_crud.pk)
                new_month_info = month_crud.add_from_model(month_info)
            else:
                month_report_id = month_or_none.pk
                month_info = MonthInfo(identity=hour2month_identity,
                                    created_time=datetime.now(),
                                    server = cur_server_pk,
                                    report_id = month_report_or_none.pk)
                new_month_info = month_crud.add_from_model(month_info)

        year_or_none = year_crud.find_one(identity=hour2year_identity,server=cur_server_pk)
        if year_or_none is None:
            year_report_or_none = year_crud.find_one(identity=hour2year_identity)
            if year_report_or_none is None:
                re_year_info = ReportYear(identity=hour2year_identity,
                                            created_time=datetime.now())
                new_re_year_crud = re_year_crud.add_from_model(re_year_info)
                year_report_id = new_re_year_crud.pk
                year_info = YearInfo(identity=hour2year_identity,
                                    created_time=datetime.now(),
                                    server = cur_server_pk,
                                    report_id = new_re_year_crud.pk)
                new_year_info = year_crud.add_from_model(year_info)
            else:
                year_report_id = year_report_or_none.pk
                year_info = YearInfo(identity=hour2year_identity,
                                    created_time=datetime.now(),
                                    server = cur_server_pk,
                                    report_id = year_report_or_none.pk)
                new_year_info = year_crud.add_from_model(year_info)


        day_info = DayInfo(identity=hour2day_identity,
                created_time=datetime.now(),
                server = cur_server_pk,
                week_id = week_report_id,
                month_id = year_report_id,
                year_id = year_report_id)
        new_day_info = day_crud.add_from_model(day_info)

        hour_singe_info = HourInfo(identity=int(index),
                                created_time=datetime.now(),
                                server=int(df_expenditure_mean.loc[index,"pk"]),
                                hdd_used_avg=int(df_expenditure_mean.loc[index,"hdd_used"]),
                                network_up_total=int(df_expenditure_sum.loc[index,"network_in"]),
                                network_down_total=int(df_expenditure_sum.loc[index,"network_out"]),
                                memory_avg=int(df_expenditure_mean.loc[index,"memory_used"]),
                                gpu_memory_avg=gpu_avg,
                                idle_rate=df_expenditure_sum.loc[index,"idle_or_not"]/df_expenditure_count.loc[index,"idle_or_not"],
                                day_id=new_day_info.pk)
        hour_crud.add_from_model(hour_singe_info)

    else:
        hour_singe_info = HourInfo(identity=int(index),
                                    created_time=datetime.now(),
                                    server=int(df_expenditure_mean.loc[index,"pk"]),
                                    hdd_used_avg=int(df_expenditure_mean.loc[index,"hdd_used"]),
                                    network_up_total=int(df_expenditure_sum.loc[index,"network_in"]),
                                    network_down_total=int(df_expenditure_sum.loc[index,"network_out"]),
                                    memory_avg=int(df_expenditure_mean.loc[index,"memory_used"]),
                                    gpu_memory_avg=gpu_avg,
                                    idle_rate=df_expenditure_sum.loc[index,"idle_or_not"]/df_expenditure_count.loc[index,"idle_or_not"],
                                    day_id=day_or_none.pk)
        hour_crud.add_from_model(hour_singe_info)

输入数据0
输入数据1


In [7]:
from database.models import DBSession

# 创建数据库连接
session = DBSession()
crud = ServerCrud(session)
server1 = Server(name="138", address="172.31.41.138")
# crud.add_from_model(server1)
# 单条件查询
server2 = crud.find_one(name="27")
# 多条件联合查询
server3 = crud.find_one(name="27", pk=1)
if server3 is not None:
    print(server3.name)
# 修改值
server3.hdd_limit = 35236
# 更新数据库
server4 = crud.update_from_model(server3)
if server4 is not None:
    print(server4.name)
session.close()

27
27
